# Нейросетевые модели поиска. Часть III. Векторный поиск.

# Домашнее задание

Цель домашнего задания - научиться строить индексы для векторного поиска на БОЛЬШИХ объемах данных, добиваясь при этом оптимального качества и скорости. Работать будем с уже известным датасетом MS MARCO (точнее, с сэмплом из него). Сэмпл генерируется следующим кодом:

In [1]:
import datasets
from datasets import load_dataset
import pandas as pd
import numpy as np

from tqdm import tqdm

In [18]:
msmarco_dataset = load_dataset("Tevatron/msmarco-passage")

Generating train split:   0%|          | 0/400782 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/6980 [00:00<?, ? examples/s]

Generating dl19 split:   0%|          | 0/43 [00:00<?, ? examples/s]

Generating dl20 split:   0%|          | 0/54 [00:00<?, ? examples/s]

In [19]:
rows = []
i = -1
j = -1
for row in tqdm(msmarco_dataset['train']):
    i += 1
    qid = i
    query = row['query']
    for pos_ex in row['positive_passages']:
        j += 1
        docid = j
        title = pos_ex['title']
        text = pos_ex['text']
        mark = 1
        rows.append([qid, query, docid, title, text, mark])
    for neg_ex in row['negative_passages']:
        j += 1
        docid = j
        title = neg_ex['title']
        text = neg_ex['text']
        mark = 0
        rows.append([qid, query, docid, title, text, mark])
df = pd.DataFrame(rows, columns=['qid', 'query', 'docid', 'title', 'text', 'mark'])

100%|██████████| 400782/400782 [03:20<00:00, 1998.61it/s]


In [20]:
df

,qid,query,docid,title,text,mark
0,0,where is whitemarsh island,0,"Whitemarsh Island, Georgia","Whitemarsh Island, Georgia. Whitemarsh Island ...",1
1,0,where is whitemarsh island,1,What military strategy was used in the pacific?,the strategy of island hopping was used by the...,0
2,0,where is whitemarsh island,2,Whakaari / White Island,"For the island near Dunedin, see White Island,...",0
3,0,where is whitemarsh island,3,Jekyll Island,"Jekyll Island, at 5,700 acres, is the smallest...",0
4,0,where is whitemarsh island,4,Sibu Island,Sibu Island. A scuba diver at Sibu Island. Sib...,0
...,...,...,...,...,...,...
12346943,400781,where is vernon tx,12346943,-,of the Lamar County. Chamber of Commerce Stree...,0
12346944,400781,where is vernon tx,12346944,-,"Honda Civic near Midland, TX; Honda Civic in L...",0
12346945,400781,where is vernon tx,12346945,-,"Driving distance from Dallas, TX to Houston, T...",0
12346946,400781,where is vernon tx,12346946,-,"Distance to cities nearby McKinney, TX and Pla...",0


In [21]:
df = df[:2000000]

In [ ]:
# df.to_csv('homework_sample.csv', index=False)

In [22]:
df

,qid,query,docid,title,text,mark
0,0,where is whitemarsh island,0,"Whitemarsh Island, Georgia","Whitemarsh Island, Georgia. Whitemarsh Island ...",1
1,0,where is whitemarsh island,1,What military strategy was used in the pacific?,the strategy of island hopping was used by the...,0
2,0,where is whitemarsh island,2,Whakaari / White Island,"For the island near Dunedin, see White Island,...",0
3,0,where is whitemarsh island,3,Jekyll Island,"Jekyll Island, at 5,700 acres, is the smallest...",0
4,0,where is whitemarsh island,4,Sibu Island,Sibu Island. A scuba diver at Sibu Island. Sib...,0
...,...,...,...,...,...,...
1999995,64923,richest man in mexico net worth,1999995,-,6. The next name on the list of richest actors...,0
1999996,64923,richest man in mexico net worth,1999996,-,[image via JJ Duncan] 1 Billy Ray Cyrus Net W...,0
1999997,64923,richest man in mexico net worth,1999997,-,"Jeff Bezos is the founder of Amazon, and one o...",0
1999998,64923,richest man in mexico net worth,1999998,-,Similar Celebrities: 1 Jeremy Mayfield Net Wo...,0


Баллы начисляются за выполнение следующих заданий:

# Предобработка данных [2 балла]

Возможно, прежде чем строить эмбеддинги из текстовых данных, стоит их предобработать? (использовать не только зону text, что-то сделать с пунктуацией, использовать зону не целиком). В случае использования только СЫРОЙ зоны text (как на семинаре) бал начислен не будет

In [23]:
import string

In [24]:
def preprocess_text(text):
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    
    # Remove words with length smaller than 4
    text = ' '.join([word for word in text.split() if len(word) >= 4])
    
    return text

In [ ]:
# Apply preprocessing to the 'text' column
df['text'] = df['text'].apply(preprocess_text)

df['text'] = df['title'] + ' ' + df['text']
df.drop('title', axis='columns', inplace=True)

In [ ]:
# df.to_csv('preprocessed_sample.csv', index=False)

In [26]:
df

,qid,query,docid,text,mark
0,0,where is whitemarsh island,0,"Whitemarsh Island, Georgia Whitemarsh Island G...",1
1,0,where is whitemarsh island,1,What military strategy was used in the pacific...,0
2,0,where is whitemarsh island,2,Whakaari / White Island island near Dunedin Wh...,0
3,0,where is whitemarsh island,3,Jekyll Island Jekyll Island 5700 acres smalles...,0
4,0,where is whitemarsh island,4,Sibu Island Sibu Island scuba diver Sibu Islan...,0
...,...,...,...,...,...
1999995,64923,richest man in mexico net worth,1999995,- next name list richest actors India that sup...,0
1999996,64923,richest man in mexico net worth,1999996,- image Duncan Billy Cyrus Worth Christina Agu...,0
1999997,64923,richest man in mexico net worth,1999997,- Jeff Bezos founder Amazon richest people wor...,0
1999998,64923,richest man in mexico net worth,1999998,- Similar Celebrities Jeremy Mayfield Worth Ji...,0


# Подбор нейронки [1 балл]

На семинаре для варки эмбеддингов использовался Universal Sentence Encoder. Возможно, стоит попробовать что-то другое? (балл будет начислен в том случае, если для варки индексов из следующих пунктов будет использована/ы другая/ие нейронки - да, использовать разные нейронки для разных индексов можно)

Для варки эмбеддингов выбор пал на [sentence-transformers/all-MiniLM-L6-v2](https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2) . Эмбеддинги на выходе имеют размерность 384, дальше она будет везде использоваться.

In [5]:
!pip install -U sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 4.2 MB/s eta 0:00:00a 0:00:01


In [6]:
from sentence_transformers import SentenceTransformer

embed = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2', device="cuda")   # убираем параметр device, если на цпу

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
text_mat = embed.encode(list(df['text'].values))
# with open('text_embs.npy', 'wb') as f:
#     np.save(f, np.array(text_mat))

In [ ]:
query_mat = embed.encode(list(df[:2000000]['query'].unique()))
# with open('query_embs.npy', 'wb') as f:
#     np.save(f, np.array(query_mat))

In [7]:
query_mat_3 = embed.encode(list(df[:300000]['query'].unique()))
# with open('query_embs_3.npy', 'wb') as f:
#     np.save(f, np.array(query_mat_1))

Batches:   0%|          | 0/304 [00:00<?, ?it/s]

# Метрика для оценки качества [1 балл]

Нужно реализовать метрику для оценки качества индексов - ndcg@10

In [2]:
import torch
from torchmetrics.retrieval import RetrievalNormalizedDCG

def NDCG(preds, target, qids, k=10):
    ndcg = RetrievalNormalizedDCG(top_k=k)
    
    return ndcg(torch.Tensor(preds),
                torch.Tensor(target),
                indexes=torch.LongTensor(qids - min(qids)))

# Варка и инференс индексов [до 6 баллов]

В рамках этой части задания нужно сварить индексы 3 типов - annoy, faiss, hnswlib. Параметры можно выбирать любые (стоит ориентироваться на качество индекса, требуемую память и перф (см. ниже).)

За 1 индекс можно получить:

    1 балл,  если индекс состоит из  300000 документов
    2 балла, если индекс состоит из 1000000 документов
    За индексы из 2000000 документов дополнительные баллы в этом пункте не начисляются, но только они могут участвовать в соревновании

Важно:

1. Эмбеддинги, которые кладутся в индекс, должны быть сгенерированы нейронкой и соответствовать документам из датасета, иначе баллы аннулируются (сложить 2 миллиона эмбеддингов вида [1,1,1,...,1] нельзя)

2. Чтобы получить баллы за индекс, необходимо не только сварить, но и проинференсить его: прогнать на нем набор запросов из датасета (~65k), оценить качество результата (используя метрику ndcg@10), оценить время инференса (напр., так как в семинаре). Без этого баллы начислены не будут

3. При варке и инференсе индекса нельзя использовать больше 16GB оперативной памяти (иначе начислится 0 баллов)

Скачаем данные (это полученные ранее данные).

In [3]:
df = pd.read_csv('/kaggle/input/vector/preprocessed_sample.csv')

with open('/kaggle/input/vector/text_embs.npy', 'rb') as f:
    text_mat = np.load(f)
with open('/kaggle/input/vector/query_embs.npy', 'rb') as f:
    query_mat = np.load(f)
with open('/kaggle/input/vector/query_embs_3.npy', 'rb') as f:
    query_mat_3 = np.load(f)

In [6]:
df

,qid,query,docid,text,mark
0,0,where is whitemarsh island,0,"Whitemarsh Island, Georgia Whitemarsh Island G...",1
1,0,where is whitemarsh island,1,What military strategy was used in the pacific...,0
2,0,where is whitemarsh island,2,Whakaari / White Island island near Dunedin Wh...,0
3,0,where is whitemarsh island,3,Jekyll Island Jekyll Island 5700 acres smalles...,0
4,0,where is whitemarsh island,4,Sibu Island Sibu Island scuba diver Sibu Islan...,0
...,...,...,...,...,...
1999995,64923,richest man in mexico net worth,1999995,- next name list richest actors India that sup...,0
1999996,64923,richest man in mexico net worth,1999996,- image Duncan Billy Cyrus Worth Christina Agu...,0
1999997,64923,richest man in mexico net worth,1999997,- Jeff Bezos founder Amazon richest people wor...,0
1999998,64923,richest man in mexico net worth,1999998,- Similar Celebrities Jeremy Mayfield Worth Ji...,0


## Annoy Index

Используется сэмпл размерности 300000

In [8]:
from annoy import AnnoyIndex

In [9]:
# build index
annoy_index = AnnoyIndex(f=384, metric='euclidean')
for i, vec in tqdm(enumerate(text_mat[:300000]), total=len(text_mat[:300000])):
    annoy_index.add_item(i, vec)

annoy_index.build(n_trees=500)
# index.save('sample500.ann')

100%|██████████| 300000/300000 [00:26<00:00, 11439.23it/s]


True

In [ ]:
# load index
# index = AnnoyIndex(f=384, metric='euclidean')
# index.load('/kaggle/input/vector/sample500.ann')

In [12]:
def eval_annoy(param=-1):
    preds = list()
    df_3 = df[:300000]

    for i, qvec in tqdm(enumerate(query_mat_3), total=len(query_mat_3)):
        filtered = df_3[df_3['qid'] == i]
        number = len(filtered)
        item = annoy_index.get_nns_by_vector(vector=qvec, n=1, search_k=param, include_distances=False)
        real_order = list(filtered['docid'])
        current_preds = [0 for _ in range(number)]
        if item[0] in real_order:
            current_preds[real_order.index(item[0])] = 1
        preds += current_preds

    return NDCG(preds, df_3['mark'].values, df_3['qid'].values)

In [13]:
%%time
eval_annoy()

100%|██████████| 9728/9728 [00:11<00:00, 826.14it/s]


CPU times: user 15.3 s, sys: 115 ms, total: 15.4 s
Wall time: 15.3 s


tensor(0.4356)

## Faiss Index

Используется сэмпл размерности 2000000

In [4]:
!pip install faiss-gpu
# !pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 18.3 MB/s eta 0:00:00:00:0100:01


In [5]:
import faiss

In [6]:
faiss_index = faiss.IndexFlatL2(384)
# faiss_index.add(text_mat)  # если работаем на цпу, то надо так подгружать данные. на гпу представлено в след ячейке

# faiss.write_index(index, "flat.index")

query_mat_2 = query_mat_2.astype(np.float32)

In [7]:
# Переведем индекс на гпу для быстрого вычисления метода search
res = faiss.StandardGpuResources()
gpu_index_flat = faiss.index_cpu_to_gpu(res, 0, faiss_index)

gpu_index_flat.add(text_mat[:2000000])

In [10]:
def eval_faiss():
    preds = list()
    
    D, I = gpu_index_flat.search(query_mat, 1)

    for i, qvec in tqdm(enumerate(query_mat), total=len(query_mat)):
        filtered = df[df['qid'] == i]
        number = len(filtered)
        real_order = list(filtered['docid'])
        current_preds = [0 for _ in range(number)]
        if I[i][0] in real_order:
            current_preds[real_order.index(I[i][0])] = 1
        preds += current_preds

    return NDCG(preds, df['mark'].values, df['qid'].values)

In [11]:
%%time
eval_faiss()

100%|██████████| 64924/64924 [01:56<00:00, 556.84it/s]


CPU times: user 4min 16s, sys: 8.76 s, total: 4min 25s
Wall time: 2min 33s


tensor(0.3780)

## hnswlib

Используется сэмпл размерности 2000000

In [13]:
!pip install hnswlib

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for hnswlib: filename=hnswlib-0.8.0-cp310-cp310-linux_x86_64.whl size=196380 sha256=976290e0c720473971282d8705376417ab0f24702ac7ca7f4018563a9ea0d84a
  Stored in directory: /root/.cache/pip/wheels/af/a9/3e/3e5d59ee41664eb31a4e6de67d1846f86d16d93c45f277c4e7
Successfully built hnswlib


In [14]:
import hnswlib

In [15]:
hnswlib_index = hnswlib.Index(space='l2', dim=384)
hnswlib_index.init_index(max_elements=2000000, ef_construction=20, M=24)

hnswlib_index.add_items(text_mat)

hnswlib_index.set_ef(ef=50)

In [16]:
def eval_hnswlib():
    preds = list()
    
    I, D = hnswlib_index.knn_query(query_mat, k=1)

    for i, qvec in tqdm(enumerate(query_mat), total=len(query_mat)):
        filtered = df[df['qid'] == i]
        number = len(filtered)
        real_order = list(filtered['docid'])
        current_preds = [0 for _ in range(number)]
        if I[i][0] in real_order:
            current_preds[real_order.index(I[i][0])] = 1
        preds += current_preds

    return NDCG(preds, df['mark'].values, df['qid'].values)

In [17]:
%%time
eval_hnswlib()

100%|██████████| 64924/64924 [02:00<00:00, 537.97it/s]


CPU times: user 4min 27s, sys: 6.54 s, total: 4min 33s
Wall time: 2min 28s


tensor(0.3075)

# Соревнование [до 10 баллов]

Проводится среди индексов размерами 2000000 документов (остальные не оцениваются) в двух дисциплинах: по качеству (величина ndcg@10) и скорости (насколько быстро удалось прогнать полный набор запросов на этом индексе)

Оценивается только один индекс (указывается сдающим домашку) - если вы сварили 3 индекса по 2 миллиона доков каждый, то укажите, какой участвует в соревновании (напишите в ноутбуке). Иначе выбор будет сделан рандомно из доступных вариантов

Разбалловка:


За первое место по качеству: 5 баллов
    
За второе место по качеству: 3 балла
    
За все остальные места по качеству, кроме последнего: 1 балл

За первое место по скорости (при не последнем месте по качеству): 5 баллов

За второе место по скорости (при не последнем месте по качеству): 3 балла

За все остальные места по скорости (при не последнем месте по качеству), кроме последнего: 1 балл

### Для соревнования выбирается второй индекс - faiss

Он оказался быстрым в поиске(за счет гпу) и более качественным. Соответсвенно проводились вычисления у него на 2000000

# Процедура сдачи

**Вам надо:**

- Форкнуть эту репу;
- Создать бранч, в котором вы дальше будете работать;
- Выполнить все или часть заданий ноутбука;
- Запушить ваш бранч и поставить Pull Request.

Проверяющий счекаутит вашу бранчу и проверит работу.